In [ ]:
import cv2
import numpy as np
import math
import open3d
from matplotlib import pyplot as plt
import copy
import os



images_folder = './N2-adults-50um-SRx5-crop-images-correct-angle-folders'
seg_folder = './N2-adults-50um-SRx5-crop-seg-correct-angle-folders'

sub_folders = os.listdir(images_folder)

print(sub_folders)

fix_tail_imags_correct_angle_folder = "N2-adults-50um-SRx5-fix-tail-crop-images-correct-angle-folders"
fix_tail_seg_correct_angle_folder = "N2-adults-50um-SRx5-fix-tail-crop-seg-correct-angle-folders"


try:
    os.makedirs(fix_tail_imags_correct_angle_folder)
    print(f"Folder '{fix_tail_imags_correct_angle_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{fix_tail_imags_correct_angle_folder}' already exists.")   


try:
    os.makedirs(fix_tail_seg_correct_angle_folder)
    print(f"Folder '{fix_tail_seg_correct_angle_folder}' created successfully.")
except FileExistsError:
    print(f"Folder '{fix_tail_seg_correct_angle_folder}' already exists.")   


for sub_folder in sub_folders:

    sub_crop_imags_folder = fix_tail_imags_correct_angle_folder + '/' + sub_folder
    sub_crop_segs_folder = fix_tail_seg_correct_angle_folder + '/' + sub_folder
    try:
        os.makedirs(sub_crop_imags_folder)
        os.makedirs(sub_crop_segs_folder)
    except FileExistsError:
        pass


In [ ]:
import math
import open3d
from copy import deepcopy


for sub_folder in sub_folders:

    
    
    print(sub_folder)

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)




    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>200:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)

    
    max_tail_y_image_index = 0
    max_tail_y = 2600
    max_tail_x = 0

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:


                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h

                if mask_y < max_tail_y:
                    max_tail_y = mask_y
                    max_tail_x = mask_x
                    max_tail_y_image_index = image_index

    print('max_tail_y_image_index, max_tail_y, max_tail_x = ',max_tail_y_image_index,max_tail_y,max_tail_x)

    
    template_centerline_by_height = {} 
    template_centerwidth_by_height = {} 

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index == max_tail_y_image_index:


                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h

                max_tail_y = mask_y
                max_tail_x = mask_x

                print(' max_tail_y, max_tail_x = ',max_tail_y,max_tail_x)

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  
                    contours_by_height[i] = filtered_data



                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])

                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x

                        r = width/2.0 

                        template_centerline_by_height[data_y] = min_x + r
                        template_centerwidth_by_height[data_y] = r

    
    slope_last = 0
    intercept_last = 0 
    slope_global = 0
    intercept_global = 0

    skip_once = False

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= (best_match_index+10):

                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 
                origin_image = cv2.imread(images_list_dir + '/' + image_name) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h


                
                crop_tail_y = mask_y + 100
                crop_tail_long_y = mask_y + 200

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  
                    contours_by_height[i] = filtered_data

                
                centerline_by_height = {}
                centerwidth_by_height = {}
                center_line = []
                center_line_long = []

                xyz_point = []
                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])
                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x
                        r = width/2.0 

                        if data_y < crop_tail_y:
                            center_line.append(((min_x + r,data_y)))

                        if data_y < crop_tail_long_y:
                            center_line_long.append(((min_x + r,data_y)))

                
                tail_points = np.array(center_line)
                
                x = np.array([point[0] for point in tail_points])
                y = np.array([point[1] for point in tail_points])
                
                slope, intercept = np.polyfit(x, y, deg=1)


                
                tail_points = np.array(center_line_long)
                
                x = np.array([point[0] for point in tail_points])
                y = np.array([point[1] for point in tail_points])
                
                slope_long, intercept_long = np.polyfit(x, y, deg=1)

                
                
                

                if abs(slope-slope_long) > 5 :

                    
                    real_slope = copy.deepcopy(slope_long)
                    real_intercept = copy.deepcopy(intercept_long)
                else:
                    real_slope = slope
                    real_intercept = intercept

                

                
                
                
                
                
                
                
                


                
                

                y_values = np.arange(max_tail_y, crop_tail_y)  
                x_values = (y_values - real_intercept) / real_slope  

                
                coordinates = np.column_stack((x_values, y_values))

                mask_copy = copy.deepcopy(mask)
                image_copy = copy.deepcopy(origin_image)


                
                for x, y in coordinates:
                    x = int(x)
                    y = int(y)
                    if r > 1:
                        r = template_centerwidth_by_height[y] - 1
                    else:
                        r = template_centerwidth_by_height[y]

                    for ix in range(int(x-r),int(x+r+1)):
                        if ix < 480 and ix > 0:
                            mask_copy[y, ix] = 255
                            image_copy[y, ix] = (215,215,215)

                cv2.imwrite(fix_tail_imags_correct_angle_folder +'/'+ sub_folder+ '/'  + image_name,image_copy)
                cv2.imwrite(fix_tail_seg_correct_angle_folder   +'/'+ sub_folder+ '/' + image_name,mask_copy)

                slope_last =  copy.deepcopy(slope)
                intercept_last =  copy.deepcopy(intercept)

In [ ]:
import math
import open3d
from copy import deepcopy


for sub_folder in sub_folders:

    if sub_folder != '6_N2_':
        continue
    print(sub_folder)

    images_list_dir = images_folder+'/'+sub_folder
    seg_images_list_dir = seg_folder+'/'+sub_folder

    images_list = os.listdir(images_list_dir)
    seg_images_list = os.listdir(seg_images_list_dir)




    
    best_match_index = 0
    best_match_score = 0
    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            if image_index==0:

                img_tempate = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

            elif image_index>200:
                img = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE)

                res = cv2.matchTemplate(img, img_tempate, cv2.TM_CCOEFF_NORMED)

                score = res[0][0]
                if score > best_match_score:
                    best_match_index = image_index
                    best_match_score = score
                    

    print('best_match_index = ',best_match_index, ', best_match_score = ' ,best_match_score)

    
    max_tail_y_image_index = 0
    max_tail_y = 2600
    max_tail_x = 0

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= best_match_index:


                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h

                if mask_y < max_tail_y:
                    max_tail_y = mask_y
                    max_tail_x = mask_x
                    max_tail_y_image_index = image_index

    print('max_tail_y_image_index, max_tail_y, max_tail_x = ',max_tail_y_image_index,max_tail_y,max_tail_x)

    
    template_centerline_by_height = {} 
    template_centerwidth_by_height = {} 

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index == max_tail_y_image_index:


                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h

                max_tail_y = mask_y
                max_tail_x = mask_x

                print(' max_tail_y, max_tail_x = ',max_tail_y,max_tail_x)

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  
                    contours_by_height[i] = filtered_data



                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])

                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x

                        r = width/2.0 

                        template_centerline_by_height[data_y] = min_x + r
                        template_centerwidth_by_height[data_y] = r

    
    slope_last = 0
    intercept_last = 0 
    slope_global = 0
    intercept_global = 0

    skip_once = False

    for image_name in images_list:

        if image_name.endswith('.png'):
            image_index = int(image_name.replace('.png',''))
            
            
            if image_index >= 0 and image_index <= (best_match_index+10):

                mask = cv2.imread(seg_images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 
                origin_image = cv2.imread(images_list_dir + '/' + image_name, cv2.IMREAD_GRAYSCALE) 

                
                h,w = mask.shape
                contours = cv2.findContours(
                    mask.astype("uint8"), cv2.RETR_CCOMP,
                    cv2.CHAIN_APPROX_NONE)[-2]

                c_index = 0
                for i in range(len(contours)):

                    mu=cv2.moments(contours[i],False)
                    area = cv2.contourArea(contours[i])
                    if area > 100000:
                        c_index = i

                mu=cv2.moments(contours[c_index],False)
                mask_center_x = int(mu['m10'] / (mu['m00']+0.00000001))
                mask_center_y = int(mu['m01'] / (mu['m00']+0.00000001))
                mask_x, mask_y, mask_w, mask_h = cv2.boundingRect(contours[c_index])
                mask_bottom_y = mask_y + mask_h


                
                crop_tail_y = mask_y + 100
                crop_tail_long_y = mask_y + 200

                
                contours_by_height = {}
                
                for i in range(h):
                    contours_by_height[i] = []

                    condition = (contours[c_index][:, :, 1] == i)  
                    filtered_data = contours[c_index][condition]  
                    contours_by_height[i] = filtered_data

                
                centerline_by_height = {}
                centerwidth_by_height = {}
                center_line = []
                center_line_long = []

                xyz_point = []
                for data_y in contours_by_height:

                    
                    length = len(contours_by_height[data_y])
                    
                    if length > 0:
                        if length == 1:
                            width = 1
                        else:

                            
                            x_coords = contours_by_height[data_y][:,0]
                            
                            max_x = np.max(x_coords)
                            min_x = np.min(x_coords)
                            width = max_x - min_x
                        r = width/2.0 

                        if data_y < crop_tail_y:
                            center_line.append(((min_x + r,data_y)))

                        if data_y < crop_tail_long_y:
                            center_line_long.append(((min_x + r,data_y)))

                
                tail_points = np.array(center_line)
                
                x = np.array([point[0] for point in tail_points])
                y = np.array([point[1] for point in tail_points])
                
                slope, intercept = np.polyfit(x, y, deg=1)


                
                tail_points = np.array(center_line_long)
                
                x = np.array([point[0] for point in tail_points])
                y = np.array([point[1] for point in tail_points])
                
                slope_long, intercept_long = np.polyfit(x, y, deg=1)

                
                print("：", image_index, slope, intercept)
                print("：", image_index, slope_long, intercept_long)

                if abs(slope-slope_long) > 5 :

                    
                    real_slope = copy.deepcopy(slope_long)
                    real_intercept = copy.deepcopy(intercept_long)
                else:
                    real_slope = slope
                    real_intercept = intercept

                

                
                
                
                
                
                
                
                


                
                

                y_values = np.arange(max_tail_y, crop_tail_y)  
                x_values = (y_values - real_intercept) / real_slope  

                
                coordinates = np.column_stack((x_values, y_values))

                mask_copy = copy.deepcopy(mask)
                image_copy = copy.deepcopy(origin_image)


                
                for x, y in coordinates:
                    x = int(x)
                    y = int(y)
                    if y < mask_y:
                        if r > 1:
                            r = template_centerwidth_by_height[y] - 1
                        else:
                            r = template_centerwidth_by_height[y]

                        for ix in range(int(x-r),int(x+r+1)):
                            if ix < 480 and ix > 0:
                                mask_copy[y, mask_x] = 255
                                image_copy[y, mask_x] = 215

                cv2.imwrite(fix_tail_imags_correct_angle_folder +'/'+ sub_folder+ '/'  + image_name,image_copy)
                cv2.imwrite(fix_tail_seg_correct_angle_folder   +'/'+ sub_folder+ '/' + image_name,mask_copy)

                slope_last =  copy.deepcopy(slope)
                intercept_last =  copy.deepcopy(intercept)